In [1]:
import os
import logging
import json
import uuid
from typing import Dict, Any, List
from dotenv import load_dotenv
from google import genai
from google.genai import types
from google.genai import errors
from abc import ABC, abstractmethod
import time
from string import Template

load_dotenv()

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

if GOOGLE_API_KEY is None:
    raise ValueError("GOOGLE_API_KEY not found in environment variables. Please set it in your .env file.")

search_products_function = types.FunctionDeclaration(
    name="search_products",
    description="Searches the product catalog for items based on a text query and optional filters.",
    parameters_json_schema={
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "A search query for products."
            },
            "filters": {
                "type": "object",
                "description": "Optional filters like color.",
                "properties": {
                    "color": {
                        "type": "string"
                    }
                }
            }
        },
        "required": ["query"]
    }
)

search_faqs_function = types.FunctionDeclaration(
    name="search_faqs",
    description="Searches a knowledge base of Frequently Asked Questions (FAQs) based on a user's query. Returns a list of relevant FAQs including their questions, answers, and categories.",
    parameters_json_schema={
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The user's question or search query to find relevant FAQs. For example: 'How long does shipping take?' or 'return policy'."
            }
        },
        "required": ["text"]
    }
)

respond_to_user_function = types.FunctionDeclaration(
    name="respond_to_user",
    description="Use this tool to send a response directly to the user. This tool also handles the formatting and routing of the message, and allows the AI to incorporate all available information to send an informative response that includes thoughts, function calls and instructions.",
    parameters_json_schema={
        "type": "object",
        "properties": {
            "content": {
                "type": "string",
                "description": "The response to be sent to the user"
            }
        },
        "required": ["content"]
    }
)

TOOLS = [
    types.Tool(function_declarations=[
        search_products_function,
        search_faqs_function,
        respond_to_user_function
    ])
]

